In [14]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
import os

In [23]:
import numpy as np
from skimage.transform import resize


def video_mamonreader(cv2,filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
    frm = resize(frame,(160,160,3))
    frm = np.expand_dims(frm,axis=0)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i +=1
    print("reading video")
    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame,(160,160,3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        i +=1
    return frames

def mamon_videoFightModel(tf,wight='mamon-videofight100.hdf5'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    input_shapes = (160,160,3)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights=None,input_shape=(100,100,3))
    for layer in base_model.layers:
        layer.trainable = False
    model = models.Sequential()
    num_classes = 2
    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())
    model = models.Sequential()
    model.add(layers.TimeDistributed(cnn,  input_shape=(40, 100, 100, 3)))
    model.add(layers.LSTM(40))
    model.add(layers.Dense(13, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(num_classes, activation="sigmoid"))
    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    model.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
    return model




def mamon_videoFightModel2(tf,wight='Users\swarna\Desktop\mini_project\Real-Time-Violence-Detection-in-Video--master\archive (1)\mamonedeffrbktoldmmon.hdf5'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    cnn = models.Sequential()
    #cnn.add(base_model)

    input_shapes=(160,160,3)
    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))
    # Freeze the layers except the last 4 layers
    #for layer in base_model.layers:
    #    layer.trainable = False

    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())
    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
    model.add(layers.LSTM(30 , return_sequences= True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = optimizers.RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model

def pred_fight(model,video,acuracy=0.9):
    pred_test = model.predict(video)
    if pred_test[0][1] >=acuracy:
        return True , pred_test[0][1]
    else:
        return False , pred_test[0][1]






In [24]:
model = mamon_videoFightModel2(tf,wight='Users\swarna\Desktop\mini_project\Real-Time-Violence-Detection-in-Video--master\archive (1)\mamonedeffrbktoldmmon.hdf5')

OSError: Unable to open file (unable to open file: name = 'Users\swarna\Desktop\mini_project\Real-Time-Violence-Detection-in-Video--masterrchive (1)\mamonedeffrbktoldmmon.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [29]:
cap = cv2.VideoCapture('hospital.mp4')
i = 0
frames = np.zeros((30, 160, 160, 3), dtype=np.float)
old = []
j = 0
while(True):
    ret, frame = cap.read()
  
    # describe the type of font
    # to be used.
    font = cv2.FONT_HERSHEY_SIMPLEX
    if i > 29:
        ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
        ysdatav2[0][:][:] = frames
        predaction = pred_fight(model,ysdatav2,acuracy=0.96)
        if predaction[0] == True:
            cv2.putText(frame, 
                'Violance Deacted  ... Violence .. violence', 
                (50, 50), 
                font, 3, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)
            cv2.imshow('video', frame)
            print('Violance detacted here ...')
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            vio = cv2.VideoWriter("./videos/output-"+str(j)+".avi", fourcc, 10.0, (fwidth,fheight))
            #vio = cv2.VideoWriter("./videos/output-"+str(j)+".mp4", cv2.VideoWriter_fourcc(*'mp4v'), 10, (300, 400))
            for frameinss in old:
                vio.write(frameinss)
            vio.release()
        i = 0
        j += 1
        frames = np.zeros((30, 160, 160, 3), dtype=np.float)
        old = []
    else:
        frm = resize(frame,(160,160,3))
        old.append(frame)
        fshape = frame.shape
        fheight = fshape[0]
        fwidth = fshape[1]
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        
        i+=1
    
    cv2.imshow('video', frame)
  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

Violance detacted here ...
Violance detacted here ...
Violance detacted here ...
Violance detacted here ...


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import time
millis = int(round(time.time() * 1000))
print("started at " , millis)
vid = video_mamonreader(cv2,'dvpalfight.dvd')
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)

In [ ]:
datav = np.zeros((1, 30, 160, 160, 3), dtype=np.float)

In [ ]:
datav[0][:][:] = vid

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,datav,acuracy=0.6))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)

In [ ]:
def video_mamonreader(cv2,filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
    frm = resize(frame,(160,160,3))
    frm = np.expand_dims(frm,axis=0)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i +=1
    print("reading video")
    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame,(160,160,3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        i +=1
    return frames

In [ ]:
novid = video_mamonreader(cv2,'nofight.mp4')

In [ ]:
nodatav = np.zeros((1, 40, 170, 170, 3), dtype=np.float)

In [ ]:
nodatav[0][:][:] = novid

In [ ]:
pred_fight(model,nodatav,acuracy=0.6)

In [6]:
ysvid2 = video_mamonreader(cv2,'hdfight.mp4')

(30, 160, 160, 3)
reading video


In [7]:
ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)

In [8]:
ysdatav2[0][:][:] = ysvid2

In [9]:
import time

millis = int(round(time.time() * 1000))
print("started at " , millis)
predaction = pred_fight(model,ysdatav2,acuracy=0.9)
print(predaction)
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)


started at  1625751803800
(True, 0.9841797)
time processing  8053


In [11]:
if predaction[0] == True:
    print('violence')

violence


In [ ]:
novid3 = video_mamonreader(cv2,'golsss.mp4')

In [ ]:
nodatav3 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
nodatav3[0][:][:] = novid3

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,nodatav3,acuracy=0.8))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)


In [ ]:

novid4 = video_mamonreader(cv2,'dvpalfight.dvd')

In [ ]:
nodatav4 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
nodatav4[0][:][:] = novid4

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,nodatav4,acuracy=0.9))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)